In [ ]:
import os
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

In [ ]:
def preprocess_image(image_path, threshold=125):
    """
    Preprocesses an image by binarizing it based on a given threshold.

    Args:
        image_path (str): Path to the image.
        threshold (int): Threshold for binarization (0-255). Defaults to 125.

    Returns:
        np.ndarray: Binarized image array.
    """
    img = mpimg.imread(image_path)
    if img.ndim == 3:  # Convert RGB to grayscale if necessary
        img = np.mean(img, axis=2)
    binary_img = (img > threshold / 255.0).astype(float)
    return binary_img

def crop_whitespace(image):
    """
    Crops the whitespace from the image by detecting the first non-white pixel 
    on all sides (top, bottom, left, right).

    Args:
        image (np.ndarray): Binarized image array.

    Returns:
        tuple: Cropping boundaries (top, bottom, left, right).
    """
    non_white_rows = np.any(image != 1, axis=1)  # Rows where any pixel is not white
    non_white_cols = np.any(image != 1, axis=0)  # Columns where any pixel is not white

    top = np.argmax(non_white_rows)
    bottom = len(non_white_rows) - np.argmax(non_white_rows[::-1]) - 1
    left = np.argmax(non_white_cols)
    right = len(non_white_cols) - np.argmax(non_white_cols[::-1]) - 1

    return top, bottom, left, right

In [ ]:
output_folder = 'cropped_images'
# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

reference_image = cv2.imread('oscii_data\images\\21 original.png', cv2.IMREAD_GRAYSCALE)
cropped_reference_image = crop_whitespace(preprocess_image("oscii_data\images\\21 original.png"))
# List of image file names to compare with the reference image
titles = ["Original Image", "AISS", "Neural Network", 
              "k-NN", "SVM", "Random Forest", 
            "k-NN no HOG", "SVM no HOG", "Random Forest no HOG", 
              "CNN", "ResNet", "MobileNetV2", "AAConverter"]
image_filenames = ['oscii_data\images\\21 original.png', 'examples\oscii_ex_catgirl\inverted_aiss.png', 'examples\oscii_ex_catgirl\inverted_nn.png',
    'examples\oscii_ex_catgirl\inverted_knn.png', 'examples\oscii_ex_catgirl\inverted_svm.png', 'examples\oscii_ex_catgirl\inverted_rforest.png', 
    'examples\oscii_ex_catgirl\inverted_knn_no_hog.png', 'examples\oscii_ex_catgirl\inverted_svm_no_hog.png', 'examples\oscii_ex_catgirl\inverted_rforest_no_hog.png', 
     'examples\oscii_ex_catgirl\inverted_cnn.png', 'examples\oscii_ex_catgirl\inverted_resnet.png', 'examples\oscii_ex_catgirl\inverted_mobile.png', 
           'examples\oscii_ex_catgirl\\aaconverter.png'
     ] 

In [ ]:
for i, filename in enumerate(image_filenames):
    # Load the original image
    original_image = mpimg.imread(filename)

    # Generate the binarized image for boundary calculation
    binarized_image = preprocess_image(filename)

    # Get cropping dimensions
    top, bottom, left, right = crop_whitespace(binarized_image)

    # Crop the original image using these dimensions
    cropped_original_image = original_image[top:bottom + 1, left:right + 1]

    # Save the cropped image in a lossless format
    output_path = os.path.join(output_folder, os.path.basename(filename))
    if cropped_original_image.max() <= 1.0:  # If the image is normalized (0-1)
        cropped_original_image = (cropped_original_image * 255).astype(np.uint8)
    cv2.imwrite(output_path, cropped_original_image)
    print(f"Cropped image saved to: {output_path}")